# Mountain Car Continuous - Hill Climbing Practice

This notebook experiments with techniques from the Udacity DRL course for hill climbing solutions to finding an optimum policy.  Code is modified from that provided in the CEM notebook provided in class.  The goal is to solve the OpenAI Gym's mountain car environment with continuous control inputs.

In [1]:
import gym
import math
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

!python -m pip install pyvirtualdisplay
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

is_ipython = 'inline' in plt.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

  Using cached https://files.pythonhosted.org/packages/ad/05/6568620fed440941b704664b9cfe5f836ad699ac7694745e7787fbdc8063/PyVirtualDisplay-2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/48/3c/75573613641c90c6d094059ac28adb748560d99bd27ee6f80cce398f404e/EasyProcess-0.3-py2.py3-none-any.whl


### Create the MountanCarContinuous environment & detect computing platform

In [2]:
env = gym.make('MountainCarContinuous-v0')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Create the agent

In [3]:
class Agent(nn.Module):
    
    # Takes in the game environment so that it can size the NN layers to match states (inputs)
    # and actions (outputs).
    # env:  the game environment (assumes OpenAI Gym API)
    # h_size:  the number of neurons in the hidden layer
    
    def __init__(self, env, h_size=16):
        super(Agent, self).__init__()
        self.env = env
        
        # state, hidden layer, action sizes
        self.s_size = env.observation_space.shape[0]
        self.h_size = h_size
        self.a_size = env.action_space.shape[0]
        
        # define layers
        self.fc1 = nn.Linear(self.s_size, self.h_size)
        self.fc2 = nn.Linear(self.h_size, self.a_size)
        
        
    # Unmarshals and stores the weights & biases in preparation for computation
    # weights:  a 1D tensor of all the weights & biases in the model; the first part of the list
    #           is all the fc1 weights, then all the fc1 biases, then all the fc2 weights
    #           and finally all the fc2 biases
    
    def set_weights(self, weights):
        s = self.s_size
        h = self.h_size
        a = self.a_size
        
        # separate the weights & biases for each layer
        fc1_end = (s+1)*h
        fc1_w = weights[: s*h].reshape(s, h)
        fc1_b = weights[s*h : fc1_end]
        fc2_w = weights[fc1_end : fc1_end + h*a].reshape(h, a)
        fc2_b = weights[fc1_end + h*a :]
        
        # set the weights for each layer
        self.fc1.weight.data.copy_(fc1_w.view_as(self.fc1.weight.data))
        self.fc1.bias.data.copy_(  fc1_b.view_as(self.fc1.bias.data))
        
        self.fc2.weight.data.copy_(fc2_w.view_as(self.fc2.weight.data))
        self.fc2.bias.data.copy_(  fc2_b.view_as(self.fc2.bias.data))
    
    
    # Returns the length of the marshalled weights list
    def get_weights_dim(self):
        return (self.s_size+1)*self.h_size + (self.h_size+1)*self.a_size
    
    
    # Performs the forward pass computation of the NN
    # x:  the vector of input data (environment states)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.tanh(self.fc2(x))
        return x.cpu().data
    
    
    # Plays one episode of the game in order to evaluate the return generated by the given policy.
    # The NN embodies the policy definition, therefore its weights encode this policy.
    # weights:  a 1D tensor of the NN's weights & biases
    # gamma:  the discount factor
    # max_t:  max number of time steps allowed in an episode
    
    def evaluate(self, weights, gamma=1.0, max_t=5000):
        self.set_weights(weights)
        episode_return = 0.0
        state = self.env.reset()
        for t in range(max_t):
            state = torch.from_numpy(state).float().to(device)
            action = self.forward(state)
            state, reward, done, _ = self.env.step(action)
            episode_return += reward * math.pow(gamma, t)
            if done:
                break
        return episode_return


### Training the model

Each of the hill climbing methods begins with a baseline policy, then samples one or more random perturbations around that baseline.  The difference is in how those samples are used to create a new baseline for the next learning iteration.

The cross-entropy method gathers many samples, then chooses the best few of these (with the highest returns on a single episode) and averages them together to form the new baseline.

The evolution method uses several samples (possibly fewer than for cross-entropy), then uses a weighted average of all of them to form the new baseline, the weighting being proportional to the return from each sample.  Thus, the samaples "higher up the hill" have more influence in moving the baseline.

In my view, cross-entropy is a generalized improvement over the evolution method; by throwing away the lower performing samples it is effectively assigning them a weight of zero.  The down-side is that by averaging the highest performing samples, it is ignoring the differences in their returns, thus possibly watering down the potential gradient ascent.  Therefore, the approach below is a hybrid of the two:  throw out any samples with reward lower than that achieved by the baseline, then use a weighted average of the remaining samples, if any.

In addition to this more aggressive hill climbing, the code uses two more techniques to help avoid getting stuck at a local maximum.
1. Using multiple random starting points within the state space
2. Adaptive noise scaling, that changes the size of the sample perturbations based on recent level of success

In [37]:
# Train the model
# agent:         the agent model to be trained
# max_episodes:  max number of episodes to train
# winning_score: the game score above which the agent is considered to be adequately trained
# gamma:         time step discount factor
# learn_rate:    multiplier for gradually adjusting the weights to the new experience
# max_restarts:  max number of restarts if a solution isn't found (starts from new random weights)
# print_every:   number of episodes between status reporting
# num_samples:   number of randomly perturbed samples to be evaluated around the baseline
# init_sigma:    the initial standard deviation of the sample perturbations
# Return:  list of scores of the baseline model from each episode

def train(agent, max_episodes=1000, winning_score=100.0, gamma=1.0, learn_rate=0.1, max_restarts=4,
          print_every=10, num_samples=10, init_sigma=0.5):
    
    SIGMA_REDUCTION = 0.99             # noise multiplier if better samples are found
    SIGMA_INCREASE  = 1.05             # noise multiplier if no better samples are found
    MIN_USABLE_SIGMA = 0.001           # lowest acceptable noise value (triggers end of training)
    MAX_USABLE_SIGMA = 1.2             # largest acceptable noise value
    MAX_EPISODES_BAD_NOISE = 9         # num epidodes we will continue to try to find a better noise sample
    MAX_EPISODES_BAD_SCORE = 12        # num episodes we will continue without seeing an improved score
    
    # loop on number of restarts
    start = 0
    solution_found = False
    while not solution_found  and  start < max_restarts:
        print("\n///// Starting new NN with random weights - start {}".format(start))
    
        # initialize bookkeeping
        scores_deque = deque(maxlen=40)
        scores = []
        adaptive_noise = True
        episodes_bad_noise = 0
        episodes_bad_score = 0
        best_score = -np.inf
    
        # get initial random weights & biases for the model and loop on episodes
        sigma = init_sigma
        min_sigma = init_sigma
        weight_size = agent.get_weights_dim()
        baseline = torch.from_numpy(sigma*np.random.randn(weight_size)/4.0) #float
        for ep in range(max_episodes):

            # evaluate the baseline and store its return
            reward = agent.evaluate(baseline, gamma=gamma)
            scores_deque.append(reward)
            scores.append(reward)
            if reward > best_score:
                best_score = reward
                episodes_bad_score = 0
            else:
                episodes_bad_score += 1

            # print status
            if ep % print_every == 0:
                bnp = baseline.numpy()
                w_avg = np.average(bnp)
                w_std = np.std(bnp)
                w_pos = 0
                for item in bnp:
                    if item > 0:
                        w_pos += 1
                pct_pos = 100.0 * w_pos / weight_size
                print("{:3} Avg score = {:6.2f}, episode = {:6.2f}, sigma = {:.3f}, " \
                      "w.avg = {:.3f}, w.std = {:.3f}, pos = {:2.0f}%"
                      .format(ep, np.mean(scores_deque), reward, sigma, w_avg, w_std, pct_pos), end='\n')

            # if average scores are high enough, end the training
            if np.mean(scores_deque) >= winning_score:
                print("\nEnvironment solved in {:d} episodes!\tAvg Score: {:.2f}".format(ep, 
                                                                                       np.mean(scores_deque)))
                solution_found = True
                break

            # else if we haven't seen an improved score for some time then end the training
            elif episodes_bad_score > MAX_EPISODES_BAD_SCORE:
                print("{:3}-{} episodes without an improved score. Best episode score = {:.2f}. Terminating."
                      .format(ep, episodes_bad_score, best_score))
                break

            # generate random samples around the baseline
            # (not clear why I need to explicitly convert both terms to float; they should be already)
            samples = torch.FloatTensor(num_samples, weight_size)
            for i in range(num_samples):
                samples[i] = baseline.float() + torch.from_numpy(sigma*np.random.randn(weight_size)).float()

            # evaluate each sample and store its return if it is better than the baseline return
            sample_rewards = []
            sample_weights = []
            for s in range(num_samples):
                r = agent.evaluate(samples[s], gamma=gamma)
                if r > reward:
                    sample_rewards.append(r)
                    sample_weights.append(samples[s])

            # if at least one sample performed better than the baseline then
            num_elite_samples = len(sample_rewards)
            if num_elite_samples > 0:
                #print("{:3} better samples".format(num_elite_samples))

                # reset the not-found counter
                episodes_without_improvement = 0

                # get the weighted average of all better samples and consider this the new baseline
                r = torch.FloatTensor(sample_rewards).resize_(num_elite_samples, 1)
                w = torch.zeros(num_elite_samples, weight_size).float()
                for s in range(num_elite_samples):
                    w[s][:] = sample_weights[s]
                baseline = (1.0-learn_rate)*baseline.float() + learn_rate*((r*w).sum(dim=0)/r.sum(dim=0)).float()

                # reduce the noise magnitude and store it as the new minimum noise
                sigma *= SIGMA_REDUCTION
                if sigma < min_sigma:
                    min_sigma = sigma

            # else (no samples performed as well as the baseline; we may have found the top of the hill)
            else:
                #print("*** No better samples found.")

                # increment the counter of no improvement
                episodes_bad_noise += 1

                # if we are still in adaptive noise phase then
                if adaptive_noise:

                    # if we haven't seen improvement for several episodes then
                    if episodes_bad_noise > MAX_EPISODES_BAD_NOISE:

                        # set noise to min used thus far and indicate transition to fine tuning phase
                        # since it appears we are close to the global maximum
                        sigma = min_sigma
                        adaptive_noise = False
                        print("{:3}-Turning off adaptive noise. Resetting sigma = {:.3f}".format(ep, sigma))

                        # reset the counter so it can be used for the fine tuning phase
                        episodes_without_improvement = 0

                    # else (still adapting the noise level)
                    else:

                        # increase the noise magnitude
                        sigma *= SIGMA_INCREASE
                        if sigma > MAX_USABLE_SIGMA:
                            sigma = MAX_USABLE_SIGMA

                # else (in fine tuning phase, near a peak)
                else:

                    # reduce the noise
                    sigma *= SIGMA_REDUCTION

                    # if we've hit the smallest noise we care to deal with then terminate
                    if sigma < MIN_USABLE_SIGMA:
                        print("{:3}-Fine tuning is at minimum noise. Best score = {:.2f}. " \
                              "Terminating search.".format(ep, best_score))
                        break

                    # increment the final phase counter and terminate after enough with no improvement
                    if episodes_bad_noise > MAX_EPISODES_BAD_NOISE:
                        print("{:3}-{} episodes of fine tuning without better noise samples. " \
                              "Best score = {:.2f}. Terminating search.".
                             format(ep, best_score, episodes_bad_noise))
                        break
                        
        start += 1
        
    return scores

In [38]:
seed = 4
env.seed(seed)
np.random.seed(seed)
agent = Agent(env).to(device)

scores = train(agent, learn_rate=0.1, init_sigma=0.5, num_samples=50, print_every=5, winning_score=90.0)


///// Starting new NN with random weights - start 0
  0 Avg score =  -0.01, episode =  -0.01, sigma = 0.500, w.avg = 0.006, w.std = 0.123, pos = 60%
  5 Avg score =  -0.00, episode =  -0.00, sigma = 0.602, w.avg = -0.000, w.std = 0.122, pos = 54%
 10 Avg score =  -0.01, episode =  -0.05, sigma = 0.724, w.avg = 0.008, w.std = 0.159, pos = 49%
Turning off adaptive noise. Resetting sigma = 0.495
11 episodes of fine tuning without better noise samples. Terminating search.

///// Starting new NN with random weights - start 1
  0 Avg score =  -0.00, episode =  -0.00, sigma = 0.500, w.avg = 0.007, w.std = 0.121, pos = 45%
  5 Avg score =  -0.00, episode =  -0.00, sigma = 0.638, w.avg = 0.007, w.std = 0.121, pos = 45%
Turning off adaptive noise. Resetting sigma = 0.500
 10 Avg score =  -0.00, episode =  -0.00, sigma = 0.500, w.avg = 0.007, w.std = 0.121, pos = 45%
11 episodes of fine tuning without better noise samples. Terminating search.

///// Starting new NN with random weights - start 2
